# **Asistente de conceptos financieros y finanzas personales con IA Generativa**

## 🤖 **Proyecto Final de IA Generativa Aplicada a la Economía**


**Curso:** Inteligencia Artificial Aplicada para la Economía  
**Estudiante:** Valentina Latorre R
**Codigo:** 201912974  

Este proyecto desarrolla un asistente de educación financiera con IA Generativa, basado en arquitectura RAG, embeddings semánticos y el modelo FLAN-T5.

##**Planteamiento del problema y objetivos**

### **¿Cuál es el problema que aborda este proyecto?**

En América Latina, una parte significativa de la población, especialmente jóvenes, trabajadores informales y pequeños emprendedores, carece de conocimientos financieros básicos. Esta falta de educación financiera se traduce en decisiones económicas deficientes, como sobreendeudamiento, bajo ahorro, desconfianza en el sistema bancario y baja participación en instrumentos de inversión. El problema se agrava por la complejidad del lenguaje técnico utilizado en el sector financiero y por la escasa oferta de herramientas accesibles y personalizadas que fomenten el aprendizaje práctico.


Este proyecto busca desarrollar un **asistente conversacional de educación financiera**, basado en inteligencia artificial generativa, que sea capaz de:
- Responder preguntas sobre conceptos financieros de forma clara, breve y adaptada al lenguaje cotidiano.
- Simular decisiones de consumo a crédito (como la compra de un celular) y ofrecer recomendaciones basadas en los ingresos del usuario.
- Facilitar el acceso al conocimiento financiero sin necesidad de habilidades técnicas ni conexión con entidades bancarias.

### **¿Cómo se relaciona este problema con la economía?**

El proyecto aborda un problema de **asimetría de información** en los mercados financieros y promueve la **eficiencia económica** mediante la toma de decisiones más informadas. También se relaciona con conceptos de **economía del comportamiento**, ya que busca mejorar la toma de decisiones individuales mediante herramientas que reducen sesgos y errores comunes.

### **¿Por qué es relevante este problema en el contexto actual?**

- Las tasas de endeudamiento informal y de sobreendeudamiento son elevadas en varios países de la región.
- Muchos jóvenes acceden a productos financieros sin comprender plenamente sus costos, lo que genera círculos de deuda.
- Con la digitalización del crédito y el consumo, la educación financiera es más urgente que nunca para evitar fraudes, malas decisiones y exclusión económica.

###**¿Por qué se aplica IA Generativa y no Predictiva?**
Porque el objetivo no es clasificar ni predecir, sino explicar y conversar.

La IA generativa puede adaptar su respuesta al contexto y a la pregunta con mayor flexibilidad que un modelo tradicional de clasificación.



In [23]:
#---------CONFIGURACIONES INICIALES---------

# --- Instalación de librerías necesarias para el proyecto ---
## Estas librerías permiten embeddings, modelos de lenguaje y la interfaz
# Se instalan las bibliotecas para procesamiento de lenguaje (transformers),
# generación de embeddings (sentence-transformers) y búsqueda vectorial (faiss).
!pip install -q faiss-cpu sentence-transformers transformers
!pip install gradio -q

# --- Importación de librerías ---
import ipywidgets as widgets
from IPython.display import display, Markdown

# --- Librerías científicas y de IA ---
import math               # Operaciones matemáticas estándar
import pandas as pd       # Manipulación y análisis de datos en tablas (DataFrame)
import torch              # Biblioteca para computación acelerada (GPU) y deep learning
import faiss              # Biblioteca de Facebook para búsqueda vectorial eficiente

# --- Librería para interfaz gráfica ---
import gradio as gr

# --- Modelos de lenguaje y embeddings ---
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [24]:
#---------CARGA Y PREPARACION DEL CORPUS---------

# --- Cargar base de conocimiento ---
# Cargar archivo CSV
df = pd.read_csv("base_conocimiento_finanzas.csv")  # Carga el archivo con temas y contenidos
df.dropna(inplace=True)                             # Elimina filas con valores vacíos
df.reset_index(drop=True, inplace=True)             # Reasigna los índices tras eliminar filas
df.head()                                           # Muestra las primeras filas para inspección

#Este corpus contiene temas y definiciones. El sistema buscará el fragmento más similar a la pregunta del usuario.


,tema,contenido
0,Que es el interes compuesto,El interes compuesto es aquel que se calcula s...
1,Como funciona una cuenta de ahorro,Una cuenta de ahorro es un producto financiero...
2,Que es un CDT,Un CDT es un certificado de deposito a termino...
3,Presupuesto personal,El presupuesto personal es una herramienta que...
4,Endeudamiento responsable,El endeudamiento responsable consiste en adqui...


In [25]:
#---------GENERACION DE EMBEDDINGS Y CREACIÓN DEL INDICE---------

# --- Embeddings e índice semántico ---
model_embed = SentenceTransformer('all-MiniLM-L6-v2')  # Cargar modelo de embeddings
embeddings = model_embed.encode(df['contenido'].tolist())  # Convertir cada contenido en un vector
index = faiss.IndexFlatL2(embeddings.shape[1])  # Crear índice FAISS basado en distancia euclidiana
index.add(embeddings)  # Agregar los vectores al índice



###**¿Qué es Word Embedding?**
Es una técnica de NLP que convierte frases en vectores numéricos que capturan su significado semántico.

In [26]:
#---------CARGAR MODELO GENERATIVO FLAN-T5---------

# --- Cargar modelo generativo para generación de respuestas ---

# Importamos las clases necesarias para cargar modelos de tipo encoder-decoder (seq2seq)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Nombre del modelo preentrenado que usaremos: FLAN-T5 en su versión base
model_name = "google/flan-t5-base"

# Cargar el tokenizador correspondiente al modelo FLAN-T5
# Este tokenizador transforma texto en tensores numéricos compatibles con el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Cargar el modelo generativo FLAN-T5
# Este modelo está optimizado para tareas de generación de texto tipo instrucción → respuesta
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


### **¿Por qué FLAN-T5 y no GPT?**
Porque FLAN-T5 está diseñado para seguir instrucciones, genera respuestas más estructuradas y es eficiente en CPU.

### **¿Qué es RAG?**
RAG = Retrieval-Augmented Generation.\
Combina búsqueda eficiente en una base de conocimiento (semántica), generando texto en lenguaje natural a partir del contexto encontrado.

**Ventaja:** El modelo no necesita memorizar todo el conocimiento; puede “buscar y razonar”. Esto reduce errores y mejora interpretabilidad.


### **¿Cómo se relaciona con conceptos de NLP y Transformers?**
- Tokenización: Se usa AutoTokenizer para convertir texto a tensores.

- Word embedding: Se generan vectores de textos completos con SentenceTransformer.

- Similitud semántica: métrica L2 (distancia euclidiana) con FAISS.

- Transformer: FLAN-T5 implementa codificador-decodificador.

- Hiperparámetros de inferencia: Para controlar la creatividad y longitud de la respuesta.

Con el contexto se limita la longitud del prompt para evitar truncamiento.



In [27]:
#---------CREAR FUNCION PRINCIPAL PARA EL AGENTE---------

# --- Agente generativo de preguntas ---
def responder_pregunta(pregunta):
    # Vectorizamos la pregunta usando el modelo de embeddings
    pregunta_vector = model_embed.encode([pregunta])

    # Buscamos el índice del fragmento más cercano (k=1 significa el más similar)
    _, I = index.search(pregunta_vector, k=1)

    # Extraemos el contenido más relevante desde el DataFrame
    contexto = df.iloc[I[0][0]]["contenido"]

    # Creamos un prompt estructurado con la información y la pregunta
    prompt = f"""
Información financiera:

{contexto}

Pregunta: {pregunta}

Respuesta en español:
"""

    # Preparamos los tensores de entrada para el modelo generativo (tokenización)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Generamos la respuesta usando el modelo FLAN-T5
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,       # Limita longitud de la respuesta
        temperature=0.7,          # Introduce algo de aleatoriedad controlada
        top_p=0.9,                # Nucleus sampling para diversidad
        do_sample=True            # Activa la aleatoriedad en la generación
    )

    # Mostrar el contexto y el prompt generado en consola
    print("📄 CONTEXTO:", contexto)
    print("🧠 PROMPT:", prompt)

    # Decodificamos la respuesta del modelo a texto legible
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



### **Prompt Engineering:**
System Prompt estructurado que incluye el contexto + la instrucción clara de responder brevemente en español.

- Instrucciones explícitas: se pide “respuesta breve en español”.

- Control de generación: Se usan hiperparámetros como ```temperature```, ```top_p```, ```max_new_tokens``` regulan creatividad y longitud.

\
**¿Por qué funciona el Prompt Engineering?**

Porque los modelos generativos siguen instrucciones mejor cuando el formato es explícito, estructurado y claro. Además, un buen prompt reduce repeticiones o respuestas genéricas.

Diferencia entre user y system prompt:\
- System prompt: se genera internamente.

- User prompt: lo que el usuario escribe, la pregunta.



In [28]:

#---------CREAR FUNCION PRINCIPAL PARA EL SIMULADOR FINANCIERO---------

# --- Simulación financiera ---
def calcular_simulacion(valor_compra, cuotas, interes_mensual, ingreso_mensual):
    tasa_decimal = interes_mensual / 100  # Convertir a porcentaje decimal

    # Fórmula de cuota mensual (anualidad ordinaria)
    if tasa_decimal == 0:
        cuota = valor_compra / cuotas
    else:
        cuota = (valor_compra * tasa_decimal) / (1 - (1 + tasa_decimal) ** (-cuotas))

    total = cuota * cuotas
    interes = total - valor_compra

    # Explicación en formato Markdown
    explicacion = (
        f"💳 **Simulación de compra**\n\n"
        f"- Cuota mensual aproximada: **${cuota:,.0f}**\n"
        f"- Total pagado al final: **${total:,.0f}**\n"
        f"- Intereses acumulados: **${interes:,.0f}**\n\n"
    )

    # Advertencia si la cuota es mayor al 40% del ingreso
    if cuota > 0.4 * ingreso_mensual:
        explicacion += "⚠️ **Advertencia:** La cuota representa más del 40% de tu ingreso mensual. Considera reducir el valor o aumentar las cuotas."
    else:
        explicacion += "✅ La cuota está en un rango manejable respecto a tus ingresos."

    return explicacion# --- Crear la interfaz con dos pestañas ---


In [29]:
#---------CREAR INTERFAZ CON GRADIO---------

# --- Crear la interfaz con dos pestañas ---
with gr.Blocks(title="Finance Assistant con IA") as demo:
    gr.Markdown("# Finance Assistant con IA – Tu asistente de finanzas personales")

    # Pestaña 1: Asistente generativo
    with gr.Tab("Asistente de Finanzas"):
        pregunta_input = gr.Textbox(label="Haz tu pregunta sobre finanzas")
        respuesta_output = gr.Textbox(label="Respuesta del asistente")
        gr.Button("Responder").click(fn=responder_pregunta, inputs=pregunta_input, outputs=respuesta_output)

    # Pestaña 2: Simulador financiero
    with gr.Tab("Simulador de Compra"):
        with gr.Row():
            valor_input = gr.Number(label="Valor compra ($)", value=800000)
            cuotas_input = gr.Slider(label="Cuotas", minimum=1, maximum=36, value=12, step=1)
        with gr.Row():
            interes_input = gr.Slider(label="Interés mensual (%)", minimum=0, maximum=5, value=2.2, step=0.1)
            ingreso_input = gr.Number(label="Ingreso mensual ($)", value=2000000)
        simulacion_output = gr.Markdown()
        gr.Button("Simular y explicar").click(
            fn=calcular_simulacion,
            inputs=[valor_input, cuotas_input, interes_input, ingreso_input],
            outputs=simulacion_output
        )

# app con URL pública
demo.launch(share=True)

#---------PRUEBAS---------

#-----Asistente de Finanzas------

#Se preguntó:¿Que es un CDT?
#Respuesta: A CDT es un certificado de deposito a termino, una inversion que se realiza por un tiempo fijo y que genera un interes definido.

#Se preguntó: ¿Que es interes ?
#Respuesta: El interes compuesto es el que se calcula sobre el capital initial y tambien sobre los intereses que se han generado en periodos anteriores.

#Se preguntó: Como invertir?
#Respuesta: As, el primer lugar de entender es que las empresas empresas de capital en el sector financiero, el cual tiene productos como CDTs, acciones, bonos o fondos mutuos de bajo riesgo, incluyen productos como CDTs, ac

#-----Simulador de Compra------

#Valor de compra: 4´500.000
#Cuotas: 24
#Interes mensual: 2,2%
#Ingreso mensual: 2´000.000
#Respuesta:

#Cuota mensual aproximada: $243,344
#Total pagado al final: $5,840,264
#Intereses acumulados: $1,340,264
# ✅ La cuota está en un rango manejable respecto a tus ingresos.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6bf6973dbdb1d34132.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **Conclusiones**

Este proyecto demuestra cómo integrar IA generativa con recuperación semántica para brindar educación financiera accesible y práctica. Se usaron modelos abiertos y gratuitos, con resultados satisfactorios en generación de texto, aunque el simulador financiero no aplica modelos de IA generativa es un buen complemento para la planificacion financiera.

### **Evaluación y validación**
Se probaron múltiples preguntas y se observó que FLAN-T5 responde de forma más concisa y clara que modelos causales. La función ```responder_pregunta``` imprime el contexto recuperado y el prompt generado, facilitando la trazabilidad. Adicional, el simulador financiero también fue validado con ejemplos reales y advertencias.

### **Posibles mejoras**

- Expansión del corpus con fuentes oficiales permitira que las respuestas sean mas precisas.


